In [ ]:
import datetime
import panel as pn
import pandas as pd
import sqlite3

from panel_full_calendar import Calendar, CalendarEvent

calendar = Calendar(
    sizing_mode="stretch_both",
    header_toolbar={
        "left": "today",
        "center": "title",
        "right": "prev,next",
    },
)
calendar.show()

In [2]:
new_years_str = "2025-01-01"
calendar.add_event(new_years_str, title="🎉 Happy New Year!")

In [3]:
marathon_dt = datetime.datetime(2025, 4, 21)
calendar.add_event(marathon_dt, title="🏃 Boston Marathon", color="red")

In [4]:
events = [
    dict(start=date, title="❌ Train")
    for date in pd.date_range(new_years_str, marathon_dt, freq="D")
]
calendar.add_events(events)

In [5]:
today = pd.to_datetime(datetime.datetime.now())

def toggle_action(event_dict):
    event = CalendarEvent.from_dict(event_dict, calendar)
    if pd.to_datetime(event.start).date() > today.date():
        return

    if "❌" in event.title:
        title = event.title.replace("❌", "✅")
    else:
        title = event.title.replace("✅", "❌")
    event.set_props(title=title)

calendar.event_click_callback = toggle_action

In [ ]:
async def update_indicators(value):
    total_days = (marathon_dt - today).days
    days_from_start = (today.date() - pd.to_datetime(new_years_str).date()).days

    # Set days_left based on total_days
    days_left.value = total_days

    completed_goals = 0
    missed_goals = 0
    current_streak = 0
    max_streak = 0
    last_date = None

    # Sort events by date to ensure chronological processing
    sorted_events = sorted(value, key=lambda x: pd.to_datetime(x["start"]).date())

    # Single loop to calculate both completed and missed goals
    for event in sorted_events:
        title = event["title"]
        if not ("✅" in title or "❌" in title):
            continue

        start = pd.to_datetime(event["start"]).date()
        # Skip future dates
        if start > today.date():
            break

        # Check for date continuity in streak
        if last_date is not None:
            days_difference = (start - last_date).days
            if days_difference > 1:  # Break in streak
                max_streak = max(max_streak, current_streak)
                current_streak = 0

        if "✅" in title:
            completed_goals += 1
            current_streak += 1
            last_date = start
        elif "❌" in title:
            missed_goals += 1
            max_streak = max(max_streak, current_streak)
            current_streak = 0
            last_date = start

    # Handle final streak
    max_streak = max(max_streak, current_streak)

    # Calculate goals met percentage if days_from_start is positive
    completion_rate.value = round(
        (max(completed_goals - 1, 0) / days_from_start * 100)
        if days_from_start > 0
        else 0
    )

    # Set current streak (use max_streak for the longest streak achieved)
    days_streak.value = max_streak

    # Set goals completed
    goals_completed.value = completed_goals

    # Set goals missed (removed the -1 as it was artificially reducing missed goals)
    goals_missed.value = max(missed_goals - 1, 0)

    # Update progress bar
    progress.value = completed_goals


progress = pn.indicators.Progress(
    value=0,
    active=False,
    max=len(events) - 2,
    sizing_mode="stretch_width",
    align="center",
)
goals_completed = pn.indicators.Number(
    value=0,
    name="Complete",
    colors=[(0, "red"), (50, "gold"), (100, "green")],
    font_size="36px",
    title_size="18px",
    align="center",
)
goals_missed = pn.indicators.Number(
    value=0,
    name="Miss",
    colors=[(0, "green"), (30, "red")],
    title_size="18px",
    font_size="36px",
    align="center",
)

completion_rate = pn.indicators.Number(
    value=0,
    name="Rate",
    format="{value}%",
    colors=[(0, "red"), (50, "gold"), (100, "green")],
    title_size="18px",
    font_size="36px",
    align="center",
)
days_streak = pn.indicators.Number(
    value=0,
    name="Streak",
    colors=[(0, "red"), (30, "gold"), (60, "green")],
    title_size="18px",
    font_size="36px",
    align="center",
)
days_left = pn.indicators.Number(
    value=0,
    name="Days left",
    colors=[(0, "red"), (60, "gold"), (120, "green")],
    title_size="18px",
    font_size="36px",
    align="center",
)
indicators = pn.Column(
    pn.pane.HTML("<h2>🚀 Progress Tracker</h2>"),
    progress,
    pn.layout.Divider(),
    pn.Row(goals_completed, goals_missed, completion_rate, align="center"),
    pn.layout.Divider(),
    pn.Row(days_streak, days_left, align="center"),
    height=500,
)

pn.bind(update_indicators, value=calendar.param.value, watch=True)
calendar.param.trigger("value")

pn.template.FastListTemplate(
    sidebar=[indicators],
    main=[calendar],
    title="📅 Marathon Training Calendar",
    accent_base_color="#D3D3D3",
    header_background="#36454F",
    theme="dark",
    sidebar_width=250,
).show()

In [ ]:
def store_events(value: list):
    if not value:
        return
    with sqlite3.connect("calendar.db") as conn:
        df = pd.DataFrame(value)
        df["start"] = df["start"].astype(str)
        df.to_sql("events", conn, if_exists="replace", index=False)

pn.bind(store_events, value=calendar.param.value, watch=True)

In [8]:
with sqlite3.connect("calendar.db") as conn:
    df = pd.read_sql("SELECT * FROM events", conn)
    if not df.empty:
        calendar.value = df.to_dict(orient="records")

In [1]:
import datetime
import panel as pn
import pandas as pd
import sqlite3

from panel_full_calendar import Calendar, CalendarEvent

# define callbacks


def store_events(value: list):
    if not value:
        return
    with sqlite3.connect("events.db") as conn:
        df = pd.DataFrame(value)
        df["start"] = df["start"].astype(str)
        df.to_sql("events", conn, if_exists="replace", index=False)


def toggle_action(event_dict: dict):
    event = CalendarEvent.from_dict(event_dict, calendar)
    if pd.to_datetime(event.start).date() > today.date():
        return

    if "❌" in event.title:
        title = event.title.replace("❌", "✅")
    else:
        title = event.title.replace("✅", "❌")
    event.set_props(title=title)


async def update_indicators(value: list):
    total_days = (marathon_dt - today).days
    days_from_start = (today.date() - pd.to_datetime(new_years_str).date()).days

    # Set days_left based on total_days
    days_left.value = total_days

    completed_goals = 0
    missed_goals = 0
    current_streak = 0
    max_streak = 0
    last_date = None

    # Sort events by date to ensure chronological processing
    sorted_events = sorted(value, key=lambda x: pd.to_datetime(x["start"]).date())

    # Single loop to calculate both completed and missed goals
    for event in sorted_events:
        title = event["title"]
        if not ("✅" in title or "❌" in title):
            continue

        start = pd.to_datetime(event["start"]).date()
        # Skip future dates
        if start > today.date():
            break

        # Check for date continuity in streak
        if last_date is not None:
            days_difference = (start - last_date).days
            if days_difference > 1:  # Break in streak
                max_streak = max(max_streak, current_streak)
                current_streak = 0

        if "✅" in title:
            completed_goals += 1
            current_streak += 1
            last_date = start
        elif "❌" in title:
            missed_goals += 1
            max_streak = max(max_streak, current_streak)
            current_streak = 0
            last_date = start

    # Handle final streak
    max_streak = max(max_streak, current_streak)

    # Calculate goals met percentage if days_from_start is positive
    completion_rate.value = round(
        (max(completed_goals - 1, 0) / days_from_start * 100)
        if days_from_start > 0
        else 0
    )

    # Set current streak (use max_streak for the longest streak achieved)
    days_streak.value = max_streak

    # Set goals completed
    goals_completed.value = completed_goals

    # Set goals missed (removed the -1 as it was artificially reducing missed goals)
    goals_missed.value = max(missed_goals - 1, 0)

    # Update progress bar
    progress.value = completed_goals


# define calendar and indicators
calendar = Calendar(
    sizing_mode="stretch_both",
    header_toolbar={
        "left": "today",
        "center": "title",
        "right": "prev,next",
    },
)

progress = pn.indicators.Progress(
    value=0,
    active=False,
    sizing_mode="stretch_width",
    align="center",
)
goals_completed = pn.indicators.Number(
    value=0,
    name="Complete",
    colors=[(0, "red"), (50, "gold"), (100, "green")],
    font_size="36px",
    title_size="18px",
    align="center",
)
goals_missed = pn.indicators.Number(
    value=0,
    name="Miss",
    colors=[(0, "green"), (30, "red")],
    title_size="18px",
    font_size="36px",
    align="center",
)

completion_rate = pn.indicators.Number(
    value=0,
    name="Rate",
    format="{value}%",
    colors=[(0, "red"), (50, "gold"), (100, "green")],
    title_size="18px",
    font_size="36px",
    align="center",
)
days_streak = pn.indicators.Number(
    value=0,
    name="Streak",
    colors=[(0, "red"), (30, "gold"), (60, "green")],
    title_size="18px",
    font_size="36px",
    align="center",
)
days_left = pn.indicators.Number(
    value=0,
    name="Days left",
    colors=[(0, "red"), (60, "gold"), (120, "green")],
    title_size="18px",
    font_size="36px",
    align="center",
)
indicators = pn.Column(
    pn.pane.HTML("<h2>🚀 Progress Tracker</h2>"),
    progress,
    pn.layout.Divider(),
    pn.Row(goals_completed, goals_missed, completion_rate, align="center"),
    pn.layout.Divider(),
    pn.Row(days_streak, days_left, align="center"),
    height=500,
)

# define initial events
new_years_str = "2025-01-01"
marathon_dt = datetime.datetime(2025, 4, 21)
today = pd.to_datetime(datetime.datetime.now())

with sqlite3.connect("events.db") as conn:
    df = pd.read_sql("SELECT * FROM events", conn)
    if not df.empty:
        # load events from database
        calendar.value = df.to_dict(orient="records")
    else:
        # or add default events
        calendar.add_event(new_years_str, title="🎉 Happy New Year!")
        calendar.add_event(marathon_dt, title="🏃 Boston Marathon", color="red")
        events = [
            dict(start=date, title="❌ Train")
            for date in pd.date_range(new_years_str, marathon_dt, freq="D")
        ]
        calendar.add_events(events)

progress.max = len(calendar.value) - 2

# set up event callbacks
calendar.event_click_callback = toggle_action
pn.bind(update_indicators, value=calendar.param.value, watch=True)
pn.bind(store_events, value=calendar.param.value, watch=True)
calendar.param.trigger("value")

# display the app
pn.template.FastListTemplate(
    sidebar=[indicators],
    main=[calendar],
    title="📅 Marathon Training Calendar",
    accent_base_color="#D3D3D3",
    header_background="#36454F",
    theme="dark",
    sidebar_width=250,
).show()

Launching server at http://localhost:56089


In [1]:
import panel as pn
from panel_full_calendar import Calendar

calendar = Calendar(
    current_date="2020-01-15",
    value=[
        {"title": "Test Event", "start": "2020-01-15"},
        {"title": "Test Event 2", "start": "2020-01-16"},
    ],
)

calendar.show()

Launching server at http://localhost:62086


In [ ]:
event = calendar.get_event_in_view("2020-01-15", title="Test Event")

event.set_props(title="New Title", all_day=False)

In [18]:
calendar.value

[{'title': 'Test Event',
  'start': '2020-01-17',
  'id': '5288253952',
  'allDay': True,
  'end': '2020-01-18'},
 {'title': 'Test Event 2', 'start': '2020-01-16', 'id': '5289305280'}]

In [10]:
import datetime
now = datetime.datetime.now()
calendar = Calendar(
    value=[
        {"title": "Drag and drop me to reschedule!", "start": now},
    ],
    editable=True,
    sizing_mode="stretch_width",
)
calendar.event_drop_callback = lambda event: print(event)

calendar.show()

Launching server at http://localhost:61359


{'oldEvent': {'allDay': False, 'title': 'Drag and drop me to reschedule!', 'start': '2025-01-16T01:27:30.382-08:00', 'id': '5288168320'}, 'event': {'allDay': False, 'title': 'Drag and drop me to reschedule!', 'start': '2025-01-15T01:27:30.382-08:00', 'id': '5288168320'}, 'relatedEvents': [], 'el': {'l': {}, 'fcSeg': {'row': 2, 'firstCol': 4, 'lastCol': 4, 'isStart': True, 'isEnd': True, 'eventRange': {'def': {'title': 'Drag and drop me to reschedule!', 'groupId': '', 'publicId': '5288168320', 'url': '', 'recurringDef': None, 'defId': '12', 'sourceId': '10', 'allDay': False, 'hasEnd': False, 'ui': {'display': None, 'constraints': [], 'overlap': None, 'allows': [], 'backgroundColor': '', 'borderColor': '', 'textColor': '', 'classNames': []}, 'extendedProps': {}}, 'ui': {'display': 'auto', 'startEditable': True, 'durationEditable': True, 'constraints': [], 'overlap': None, 'allows': [], 'backgroundColor': 'null', 'borderColor': 'null', 'textColor': 'null', 'classNames': []}, 'instance': {

In [8]:
calendar.value

[{'title': 'Test Event 2', 'start': '2020-01-16', 'id': '6050618560'}]